In [1]:
from bokeh.io import output_notebook
import pandas as pd
import numpy as np

output_notebook()

Loading BokehJS ...

In [2]:
file = "../data/csv_files/blue_jays.csv"

df = pd.read_csv(file)
df.head()

,BirdID,KnownSex,BillDepth,BillWidth,BillLength,Head,Mass,Skull,Sex
0,0000-00000,M,8.26,9.21,25.92,56.58,73.30,30.66,1
1,1142-05901,M,8.54,8.76,24.99,56.36,75.10,31.38,1
2,1142-05905,M,8.39,8.78,26.07,57.32,70.25,31.25,1
3,1142-05907,F,7.78,9.30,23.48,53.77,65.50,30.29,0
4,1142-05909,M,8.71,9.84,25.47,57.32,74.90,31.85,1


In [3]:
df["skull_size"] = [2 * (2 ** (size / 10)) for size in df["Skull"]]

df.head()

,BirdID,KnownSex,BillDepth,BillWidth,BillLength,Head,Mass,Skull,Sex,skull_size
0,0000-00000,M,8.26,9.21,25.92,56.58,73.30,30.66,1,16.748965
1,1142-05901,M,8.54,8.76,24.99,56.36,75.10,31.38,1,17.606058
2,1142-05905,M,8.39,8.78,26.07,57.32,70.25,31.25,1,17.448124
3,1142-05907,F,7.78,9.30,23.48,53.77,65.50,30.29,0,16.324875
4,1142-05909,M,8.71,9.84,25.47,57.32,74.90,31.85,1,18.189072


In [4]:
male = df[df["Sex"] == 1]
female = df[df["Sex"] == 0]

In [5]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import Legend


def plot_scatter(df, fill_color=None, text=None):
    p = figure(
        title="Figure 12.3",
        height=400,
        width=400,
        y_range=(50, 62),
        x_axis_label="body mass (g)",
        y_axis_label="head length (mm)",
        toolbar_location=None,
    )

    p.scatter(
        x="Mass",
        y="Head",
        size="skull_size",
        source=df,
        fill_color=fill_color,
        line_color="white",
    )
    p.yaxis.ticker = [52, 54, 56, 58, 60]

    # change headline text
    p.title.text = text

    # style the headline
    p.title.text_font_size = "12px"
    p.title.align = "center"
    p.title.background_fill_color = "darkgrey"
    p.title.background_fill_alpha = 0.5
    p.title.text_color = "black"

    legend = Legend(
        items=[("skull_size", [p])],
        location="bottom_right",
        orientation="horizontal",
        border_line_color=None,
        title="skull size (mm)",
    )
    p.add_layout(legend)

    return p

In [6]:
a = plot_scatter(female, "#DF7307", "female birds")
b = plot_scatter(male, "#2769AB", "male birds")

layout = gridplot([[a, b]])
show(layout)

ValueError: failed to validate Legend(id='p1044', ...).items: expected an element of List(Instance(LegendItem)), got seq with invalid items [('skull_size', [figure(id='p1001', ...)])]